In [24]:
import pandas as pd
import matplotlib.pyplot as plt

# 한글화
plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False

In [25]:
# 1. 데이터 불러오기
df = pd.read_csv('한국_기업문화_HR_데이터셋_샘플.csv')

# 2. 데이터 확인
print(df.head())

   Unnamed: 0  Age 이직여부           출장빈도  일일성과지표                      부서  집까지거리  \
0        1041   28   No  Travel_Rarely     866                   Sales      5   
1         184   53   No  Travel_Rarely    1084  Research & Development     13   
2        1222   24  Yes  Travel_Rarely     240         Human Resources     22   
3          67   45   No  Travel_Rarely    1339  Research & Development      7   
4         220   36   No  Travel_Rarely    1396  Research & Development      5   

   학력수준             전공분야  EmployeeCount  ...  대인관계만족도  StandardHours 스톡옵션등급  \
0     3          Medical              1  ...        4             80      0   
1     2          Medical              1  ...        3             80      2   
2     1  Human Resources              1  ...        3             80      1   
3     3    Life Sciences              1  ...        3             80      1   
4     2    Life Sciences              1  ...        4             80      0   

   총경력  연간교육횟수  워라밸 현회사근속년수  현직무근속년수 최

In [26]:
# 3단계. 칼럼명 한글에서 영어로 변경 
df.rename(columns={
    '이직여부': 'Attrition',
    '야근여부': 'OverTime',
    '업무만족도': 'JobSatisfaction',
    '워라밸': 'WorkLifeBalance',
    '집까지거리': 'DistanceFromHome',
    '현회사근속년수': 'YearsAtCompany',
    '출장빈도': 'BusinessTravel',
}, inplace=True)

In [28]:
# 4단계. 칼럼 선별 6개 (이직율과 가장 높은 상관관계 6개 선정)

df_selected = df[['Attrition', #  예측 대상 변수 (이직 여부: Yes/No)
                  'OverTime', #  야근 여부 –> 스트레스, 이직률과 매우 높은 상관
                  'JobSatisfaction', #  업무 만족도 –> 낮을수록 이직 가능성↑
                  'WorkLifeBalance',  #  워라밸 만족도 –> 조직문화·불만족 요인
                  'DistanceFromHome',  #  집과 직장 간 거리 –> 통근 피로도는 주요 이직 사유
                  'YearsAtCompany', #  현 회사 근속 연수 –> 짧을수록 이직률 높음
                  'BusinessTravel']]   #  출장 빈도 –> 잦은 출장은 소진과 이직에 영향

# 확인
print(df_selected.head())


  Attrition OverTime  JobSatisfaction  WorkLifeBalance  DistanceFromHome  \
0        No       No                1                3                 5   
1        No       No                1                3                13   
2       Yes       No                3                3                22   
3        No       No                1                3                 7   
4        No       No                2                4                 5   

   YearsAtCompany BusinessTravel  
0               5  Travel_Rarely  
1               4  Travel_Rarely  
2               1  Travel_Rarely  
3               1  Travel_Rarely  
4              13  Travel_Rarely  


In [29]:
# 5단계 머신러닝을 위한 데이터 변환 (범주형 변수 -> 숫자형 변수 변환)
from sklearn.preprocessing import LabelEncoder

# 야근빈도와 이직여부는 이진범주라 LabelEncoding
df_selected['Attrition'] = df_selected['Attrition'].map({'Yes': 1, 'No': 0})
df_selected['OverTime'] = df_selected['OverTime'].map({'Yes': 1, 'No': 0})

# 출장은 One-Hot 정수형 변환
df_selected = pd.get_dummies(df_selected, columns=['BusinessTravel'], drop_first=True, dtype=int)

# 결과 확인
print(df_selected.head())

   Attrition  OverTime  JobSatisfaction  WorkLifeBalance  DistanceFromHome  \
0          0         0                1                3                 5   
1          0         0                1                3                13   
2          1         0                3                3                22   
3          0         0                1                3                 7   
4          0         0                2                4                 5   

   YearsAtCompany  BusinessTravel_Travel_Frequently  \
0               5                                 0   
1               4                                 0   
2               1                                 0   
3               1                                 0   
4              13                                 0   

   BusinessTravel_Travel_Rarely  
0                             1  
1                             1  
2                             1  
3                             1  
4                             

C:\Users\chlwo\AppData\Local\Temp\ipykernel_94440\3773233403.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['Attrition'] = df_selected['Attrition'].map({'Yes': 1, 'No': 0})
C:\Users\chlwo\AppData\Local\Temp\ipykernel_94440\3773233403.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['OverTime'] = df_selected['OverTime'].map({'Yes': 1, 'No': 0})


In [30]:
# 6단계 데이터 전처리(결측치 제거)
print(df_selected.isnull().sum())
# 이미 결측치가 없으니 이상치만 보자.

Attrition                           0
OverTime                            0
JobSatisfaction                     0
WorkLifeBalance                     0
DistanceFromHome                    0
YearsAtCompany                      0
BusinessTravel_Travel_Frequently    0
BusinessTravel_Travel_Rarely        0
dtype: int64


In [32]:
# 7단계 데이터 전처리(이상치 처리)

# IQR 방식으로 이상치 확인 
# 코드 출처 : https://velog.io/@jihyunko/%EC%9D%B4%EC%83%81%EC%B9%98-%EC%89%BD%EA%B2%8C-%EC%B2%98%EB%A6%AC%ED%95%98%EA%B8%B0

def findOutliers(x, column):
    q1 = x[column].quantile(0.25)
    q3 = x[column].quantile(0.75)
    iqr = q3 - q1
    y = x[(x[column] > (q3 + 1.5 * iqr)) | (x[column] < (q1 - 1.5 * iqr))]
    return y  # 이상치만 반환

# 예: DistanceFromHome 컬럼의 이상치 확인
outliers_df = findOutliers(df_selected, 'DistanceFromHome')
print("이상치 개수:", len(outliers_df))
print(outliers_df[['DistanceFromHome']])

# 이상치도 딱히 없다는 것을 확인

이상치 개수: 0
Empty DataFrame
Columns: [DistanceFromHome]
Index: []


In [33]:
# 8단계 모델 훈련 
# 학습/검증 데이터를 8:2 비율로 분할하시오.
# pandas -> numpy 변환

import numpy as np
from sklearn.model_selection import train_test_split

# X: 독립 변수, y: 종속 변수
X = df_selected.drop('Attrition', axis=1).values 
y = df_selected['Attrition'].values               

# 학습/검증 데이터 분할 (8:2 비율)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 확인
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train 이직자 비율:", round(np.mean(y_train), 4))
print("y_test 이직자 비율:", round(np.mean(y_test), 4))




X_train shape: (800, 7)
X_test shape: (200, 7)
y_train 이직자 비율: 0.1575
y_test 이직자 비율: 0.16


In [34]:
# 9단계: LogisticRegression 모델 학습 및 평가
# 출처: [sklearn 기본] 승객 생존율 예측 https://blog.naver.com/atomickimm/222298669290

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# 모델 정의 및 학습
log_reg = LogisticRegression(random_state=13, solver='liblinear', C=10.0)
log_reg.fit(X_train, y_train)

# 예측
pred = log_reg.predict(X_test)

# 정확도 출력
print("정확도:", round(accuracy_score(y_test, pred), 4))

# 혼동 행렬 출력
print("혼동 행렬:\n", confusion_matrix(y_test, pred))

# 분류 성능 상세 지표
print("\n 상세지표:\n", classification_report(y_test, pred))

정확도: 0.825
혼동 행렬:
 [[164   4]
 [ 31   1]]

 상세지표:
               precision    recall  f1-score   support

           0       0.84      0.98      0.90       168
           1       0.20      0.03      0.05        32

    accuracy                           0.82       200
   macro avg       0.52      0.50      0.48       200
weighted avg       0.74      0.82      0.77       200



결과를 해석하자면, 정확도는 굉장히 높게 나왔으나,

이직자의 recall값이 너무 낮게 나왔다. 즉, 실패한 모델이다.
0.03이 이직자로 보고 예측한 경우가 3%밖에 안된다는 뜻이다.
시간이 없어서 수정은 못하나, 모델이 학습할 때 0의 패턴에 너무 민감하게 반응하는게 가장 큰 원인이 아닐까 싶다.

In [36]:
# 10단계 : 예측 결과 분석
# 이직여부 = YES로 예측된 직원수를 출력하시오
pred = log_reg.predict(X_test)
num_attrition_yes = np.sum(pred == 1)
print("이직여부가 YES로 예측된 직원 수:", num_attrition_yes)


이직여부가 YES로 예측된 직원 수: 5


In [37]:
#이직 가능성이 높다고 판단되는 상위 5명에 대한 데이터를 출력하시오.
top_5_attrition = X_test[pred == 1][:5]
print("\n이직 가능성이 높다고 판단되는 상위 5명 데이터:\n", top_5_attrition)


이직 가능성이 높다고 판단되는 상위 5명 데이터:
 [[ 1  1  2 16  2  0  1]
 [ 1  4  1 26  1  1  0]
 [ 1  1  3 28  3  0  1]
 [ 1  1  2 16  0  0  1]
 [ 1  1  2  3  1  0  1]]


In [ ]:
import pandas as pd 
# 출처 : 기존 제가 케글에서 사용한 코드를 활용했습니다.
# Step 1. 신규 입사자 데이터 입력 (편집)
신입사원_데이터 = [
    { "OverTime": "Yes", "JobSatisfaction": 2, "WorkLifeBalance": 2,
    "DistanceFromHome": 5, "YearsAtCompany": 1, "BusinessTravel": "Travel_Rarely"
    },{"OverTime": "No", "JobSatisfaction": 4, "WorkLifeBalance": 3,
        "DistanceFromHome": 10, "YearsAtCompany": 7, "BusinessTravel": "Non-Travel"
    },{ "OverTime": "Yes", "JobSatisfaction": 1, "WorkLifeBalance": 2, "DistanceFromHome": 2, "YearsAtCompany": 2, "BusinessTravel": "Travel_Frequently"
    }
]
df_new = pd.DataFrame(신입사원_데이터)
# 범주형 → 숫자형
df_new['OverTime'] = df_new['OverTime'].map({'Yes': 1, 'No': 0})
# 아래는 위와 같은 처리 방식
df_new = pd.get_dummies(df_new, columns=['BusinessTravel'])
for col in ['BusinessTravel_Travel_Frequently', 'BusinessTravel_Travel_Rarely']:
    if col not in df_new.columns:
        df_new[col] = 0
columns_used = ['OverTime', 'JobSatisfaction', 'WorkLifeBalance','DistanceFromHome', 'YearsAtCompany', 'BusinessTravel_Travel_Frequently', 'BusinessTravel_Travel_Rarely']
df_new = df_new[columns_used]

# Step 7. 예측 수행
predictions = log_reg.predict(df_new)
print("신규 입사자 이직 예측 결과 (0=잔류, 1=이직):", predictions)

신규 입사자 이직 예측 결과 (0=잔류, 1=이직): [0 0 1]


c:\anaconda\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
